In [56]:
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import matplotlib.pyplot as plt
import json
import csv
import pandas
from pandas.io.json import json_normalize  

In [85]:
with open('output/losangeles.eventful.com!events!categories!music.json') as f: 
    d = json.load(f) 
    df = json_normalize(d["elements"]) 
    df.drop(labels="textContent", axis=1)
    df.to_csv("output/losangeles.eventful.com!events!categories!music.csv")
    print(df.columns) 

Index(['top', 'left', 'width', 'height', 'itemType', 'itemProp', 'tagName',
       'elementId', 'classList', 'fontSize', 'fontStyle', 'font', 'fontWeight',
       'lineHeight', 'opacity', 'color', 'backgroundColor', 'backgroundImage',
       'background', 'textContent'],
      dtype='object')
